# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!python --version

Python 3.7.1


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
#!pip install ipywidgets

from dotenv import load_dotenv
import os
import json
from pprint import pprint
load_dotenv()
API_KEY=os.getenv('google_api_key')
#!pip install gmaps
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter nbextension enable --py widgetsnbextension
#!jupyter nbextension enable --py --sys-prefix gmaps
#from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
vacationdf=pd.read_csv('weatherdf.csv')
vacationdf

,Unnamed: 0,city,lattitude,longitude,max_temp,humidity,cloudiness,wind_speed,country,date
0,0,Arraial do Cabo,-22.9661,-42.0278,73.36,88,75,16.11,BR,1630064268
1,1,Pevek,69.7008,170.3133,38.52,75,100,8.90,RU,1630064269
2,2,Poum,-20.2333,164.0167,73.94,78,56,14.07,NC,1630064269
3,3,Tasiilaq,65.6145,-37.6368,50.16,97,100,9.64,GL,1630064269
4,4,Vila Franca do Campo,37.7167,-25.4333,70.21,86,94,11.12,PT,1630064270
...,...,...,...,...,...,...,...,...,...,...
571,571,Shīeli,44.1667,66.7500,97.79,11,27,9.78,KZ,1630064461
572,572,New Glasgow,45.5834,-62.6486,72.10,83,3,8.55,CA,1630064462
573,573,Phan Thiet,10.9333,108.1000,77.65,87,98,3.40,VN,1630064462
574,574,Nur-Sultan,51.1801,71.4460,85.95,23,0,11.18,KZ,1630064462


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure GMAPS
gmaps.configure(API_KEY)


In [10]:
humidity=vacationdf['humidity'].astype(float)
max_humidity=humidity.max()
city=vacationdf[['lattitude','longitude']]
fig=gmaps.figure(center = [31.9686, -99.9018], zoom_level = 3)
heat_layer=gmaps.heatmap_layer(city,weights=humidity,max_intensity=max_humidity,dissipating=False,point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

0      88.0
1      75.0
2      78.0
3      97.0
4      86.0
       ... 
571    11.0
572    83.0
573    87.0
574    23.0
575    78.0
Name: humidity, Length: 576, dtype: float64

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
best=vacationdf.loc[(vacationdf['max_temp']<75)&(vacationdf['max_temp']>70)&(vacationdf['cloudiness']<10)]
best

,Unnamed: 0,city,lattitude,longitude,max_temp,humidity,cloudiness,wind_speed,country,date
56,56,Guerrero Negro,27.9769,-114.0611,72.84,80,0,4.27,MX,1630064287
136,136,Ukiah,39.1502,-123.2078,72.03,65,1,0.00,US,1630064086
227,227,Matamoros,25.5333,-103.2500,70.11,65,5,5.30,MX,1630064344
237,237,Shunyi,40.1250,116.6453,73.35,60,0,4.47,CN,1630064348
281,281,Panjāb,34.3880,67.0233,73.42,11,5,9.06,AF,1630064362
385,385,Russell,32.3502,-85.1999,74.91,90,1,3.44,US,1630064398
425,425,Camargo,27.6667,-105.1667,70.66,62,6,4.14,MX,1630064414
477,477,Jiménez,28.3333,-105.4000,70.61,61,9,2.91,MX,1630064431
494,494,Port Hedland,-20.3167,118.5667,73.45,78,0,8.05,AU,1630064437
516,516,Laguna,38.4210,-121.4238,73.76,67,1,3.44,US,1630064444


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df=best
hotel_df['hotel_name']=''

C:\Users\joelw\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
radius=5000
target_type='lodging'

In [28]:
base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'
hlist=len(set(hotel_df))

for index,row in hotel_df.iterrows():
    locs=hotel_df['lattitude'][row].astype(str)+','+hotel_df['longitude'][row].astype(str)
    params={
        'location':target_coordinates,
        'radius': target_radius,
        'type':target_type,
        'key':API_KEY
    }
    response=requests.get(base_url,params=params)
    locations=response.json()
    
hotel_df['hotel_name'][row]=locations['results'][0]['name']
hotel_df

NameError: name 'target_coordinates' is not defined

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
